In [1]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch.onnx
import torch
import onnx
import numpy as np
import onnxruntime as ort
from workload_generator import parse_onnx, write_csv



/home/jeffjw/anaconda3/envs/DWG/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:

# Load the pretrained model and tokenizer
model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Attach hook
def store_input_hook(module, input, output):
    module.input_hooked = input

model = GPT2LMHeadModel.from_pretrained(model_name)
hooked = {}
# Do a recursive search for all modules that are of type "lookup_modules" and register the hook
def register_hook(module, hook, lookup_modules = [torch.nn.Linear]):
    for child in module.children():
        if isinstance(child, tuple(lookup_modules)):
            handle = child.register_forward_hook(store_input_hook)
            hooked[child] = handle
        else:
            register_hook(child, store_input_hook, lookup_modules)

register_hook(model, hooked, lookup_modules = [torch.nn.Linear, type(model.transformer.h[0].mlp.c_fc)])

print(hooked)


# Example usage
input_text = "Hello, how are you?"
input_ids = tokenizer.encode(input_text, return_tensors="pt")
output = model.generate(input_ids)
decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)

print(decoded_output)


# Set the path for saving the ONNX file
onnx_file_path = "model.onnx"

# Set the input tensor
input_ids = torch.tensor([[15496, 11, 703, 389, 345, 30]])

# Export the model to ONNX
torch.onnx.export(model, input_ids, onnx_file_path)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


{Conv1D(): <torch.utils.hooks.RemovableHandle object at 0x7f30712afdc0>, Conv1D(): <torch.utils.hooks.RemovableHandle object at 0x7f306fd3fac0>, Conv1D(): <torch.utils.hooks.RemovableHandle object at 0x7f30712a3040>, Conv1D(): <torch.utils.hooks.RemovableHandle object at 0x7f30712a3130>, Conv1D(): <torch.utils.hooks.RemovableHandle object at 0x7f30712a3760>, Conv1D(): <torch.utils.hooks.RemovableHandle object at 0x7f30712a3610>, Conv1D(): <torch.utils.hooks.RemovableHandle object at 0x7f30712a3490>, Conv1D(): <torch.utils.hooks.RemovableHandle object at 0x7f30712a3340>, Conv1D(): <torch.utils.hooks.RemovableHandle object at 0x7f30712a3250>, Conv1D(): <torch.utils.hooks.RemovableHandle object at 0x7f30712a3940>, Conv1D(): <torch.utils.hooks.RemovableHandle object at 0x7f30712a3d90>, Conv1D(): <torch.utils.hooks.RemovableHandle object at 0x7f30712a3ac0>, Conv1D(): <torch.utils.hooks.RemovableHandle object at 0x7f30712a34c0>, Conv1D(): <torch.utils.hooks.RemovableHandle object at 0x7f3071

/home/jeffjw/anaconda3/envs/DWG/lib/python3.8/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Hello, how are you?

I'm a little bit of a nerd. I'm a


In [30]:
keys = [k.input_hooked[0] for k in hooked.keys()]

In [46]:
in_A = keys[0][0,0]
in_B = list(hooked.keys())[0].weight[:,:4]

In [42]:
ref_out = in_A @ in_B

In [44]:
ref_out

tensor([-0.0102,  0.4479,  0.9823, -0.4952], grad_fn=<SqueezeBackward4>)

In [ ]:
# Load the ONNX model
onnx_model = onnx.load("model.onnx")


In [ ]:
# Parse the ONNX model
parsed_model, other_ops = parse_onnx(onnx_model)

In [ ]:
pdict = {k: v for k,v in parsed_model.items() if v.op_type == 'Gemm' or v.op_type == 'MatMul'}
pdict['transformer.h.1.attn.MatMul']

In [ ]:
# Write the parsed model to a CSV file
write_csv(parsed_model, "gpt2_gen.csv")

In [ ]:
import torch 
